<a href="https://colab.research.google.com/github/remphan1618/liteOutInpainta/blob/main/IOPAINT%20FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#@title Hugging Face Login & Model Lister

import ipywidgets as widgets
from IPython.display import display
import huggingface_hub
import os
from huggingface_hub.utils import HfHubHTTPError # Import for better error handling

# --- Section 1: Token Input Widget ---
# We need to create the widget *outside* any conditional logic
# so it's displayed when the cell is run the first time.

# Display instructions
print("Step 1: Please enter your Hugging Face token in the box below.")
print("You can get your token from your Hugging Face account settings:")
print("https://huggingface.co/settings/tokens")
print("\nStep 2: After pasting the token, **run this cell again**.")


# Check if a widget with this description already exists to avoid displaying duplicates
# This is a common pattern in notebooks when re-running cells
try:
    # Attempt to find the widget by its description
    token_input # Check if the variable exists from a previous run
    print("\nUsing existing HF Token input box.")
except NameError:
    # Create the widget if it doesn't exist
    token_input = widgets.Text(
        value='',
        placeholder='Paste your HF token here',
        description='HF Token:',
        disabled=False,
        password=True # Masks the input for security
    )
    # Display the widget only if we just created it
    display(token_input)


# --- Section 2: Login and Model Listing (Runs on cell execution) ---

# Access the token value from the widget *when the cell is executed*
user_token = token_input.value.strip() # Use strip() to remove potential leading/trailing whitespace

# Check if a token was actually provided in the input box
if user_token:
    print("\nAttempting to log in to Hugging Face...")
    try:
        # Use huggingface_hub.login() which handles setting up the environment
        # We pass the token directly from the widget value
        huggingface_hub.login(token=user_token)
        print("Successfully logged in to Hugging Face.")

        # --- Section 3: Model Listing (Only runs after successful login) ---
        print("\nProceeding to list models...")

        # Define the filters
        try:
            # This call will now use the credentials set up by huggingface_hub.login()
            models_generator = list_models(
                pipeline_tag="text-to-image",
                library="diffusers",
                search="inpaint",
                sort="lastModified", # Sort by last update time
                direction=-1,       # -1 for descending (newest first)
                # The token parameter is usually not needed here if login() was successful,
                # but you could explicitly pass it: token=user_token
            )

            # Extract the repo_ids
            # Convert generator to a list if needed for repeated access,
            # but for your printing logic, iterating the generator directly works.
            inpaint_model_ids = [model.modelId for model in models_generator]


            # --- Print model IDs 10 per line, space-separated ---
            print("\nList of Inpaint Model IDs (10 per line, space-separated):")
            if inpaint_model_ids:
                # Print model IDs in blocks of 10 per line
                for i in range(0, len(inpaint_model_ids), 10):
                    # Get a slice of 10 model IDs
                    batch = inpaint_model_ids[i:i + 10]
                    # Join them with a space and print
                    print(" ".join(batch))
            else:
                print("No models found matching the criteria.")

            # --- Print model IDs in chunks, comma-separated with trailing comma ---
            comma_chunk_size = 5 # You can change this to 10 if you prefer
            print(f"\nComma-separated list ({comma_chunk_size} per line, with space and trailing comma):")
            if inpaint_model_ids:
                # Print model IDs in blocks of 'comma_chunk_size' per line
                for i in range(0, len(inpaint_model_ids), comma_chunk_size):
                    # Get a slice of 'comma_chunk_size' model IDs
                    batch = inpaint_model_ids[i:i + comma_chunk_size]
                    # Join them with a comma and a space
                    line = ", ".join(batch)

                    # Check if this is the last batch
                    is_last_batch = (i + comma_chunk_size) >= len(inpaint_model_ids)

                    if not is_last_batch:
                        # If not the last batch, print the line followed by a comma and newline
                        print(line + ",") # Add trailing comma
                    else:
                        # If it is the last batch, just print the line
                        print(line)
            else:
                 print(f"\nNo models found to create a comma-separated list ({comma_chunk_size} per line).")


        except HfHubHTTPError as e:
            print(f"\nAn HTTP error occurred during model listing: {e}")
            print("Please check your filters and ensure the token has read permissions.")
        except Exception as e:
            print(f"\nAn unexpected error occurred during model listing: {e}")

    except Exception as e:
        # Catch login errors specifically
        print(f"\nHugging Face login failed: {e}")
        print("Please check your token and try again.")
        print("Make sure the token has sufficient permissions (at least 'read').")

else:
    # This block runs if user_token is empty (typically on the first execution)
    print("\nNo token detected in the input box.")
    print("Please paste your token above and run this cell again to log in and list models.")

# You can optionally unset the widget's value for added security after processing
# token_input.value = '' # Uncomment if you want to clear the box after use

An HTTP error occurred: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models?filter=diffusers&pipeline_tag=text-to-image&search=inpaint&sort=lastModified&direction=-1 (Request ID: Root=1-681169d6-46b72a3a39d9567348f59ee0;9d40ccda-5787-4064-8f9f-93cda1f922b2)

Invalid credentials in Authorization header


In [ ]:
#@title 🎨 All-in-One IOpaint with Ngrok

#@markdown ## 🔑 Ngrok Configuration
#@markdown Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken) below:
ngrok_token = "2tjxIXifSaGR3dMhkvhk6sZqbGo_6ZfBZLZHMbtAjfRmfoDW5" #@param {type:"string"}
save_token = True #@param {type:"boolean"}
token_file = "/content/ngrok_token" #@param {type:"string"}

#@markdown ## 📦 Model Selection
#@markdown ### Checkpoint Models (comma-separated list)
checkpoint_models = "lama, sam2_base, gfpgan" #@param {type:"string"}

#@markdown ### Diffusers Models (comma-separated list of HF repos)
diffusers_models = "briaai/BRIA-2.3-Inpainting, Peixia/sdxl_inpainting_controlnet, Whenzi/Inpaint_carrot_crack, mrcuddle/URPM-Inpaint-Hyper-SDXL, mrcuddle/URPM-Inpaint-SDXL, andro-flock/LUSTIFY-SDXL-NSFW-checkpoint-v2-0-INPAINTING, andro-flock/Clarity-Clarity-v3-inpainting, andro-flock/Liberty-LibertyMain-Inpainting," #@param {type:"string"}
use_fp16_for_diffusers = True #@param {type:"boolean"}
pre_download_diffusers = True #@param {type:"boolean"}

#@markdown ## 🚀 IOpaint Options
selected_model = "urpm_inpaint_sdxl" #@param ["lama", "mat", "migan", "sdxl_turbo", "urpm_inpaint_sdxl"]
enable_interactive_seg = True #@param {type:"boolean"}
enable_remove_bg = False #@param {type:"boolean"}
enable_realesrgan = False #@param {type:"boolean"}
enable_gfpgan = True #@param {type:"boolean"}
disable_nsfw = True #@param {type:"boolean"}
port = 8080 #@param {type:"integer"}

import os
import sys
import json
import time
import asyncio
import requests
from datetime import datetime
from tqdm.notebook import tqdm
from IPython.display import clear_output, display, HTML

# Current timestamp
print(f"Current Date: 2025-04-29 22:10:41 UTC")
print(f"User: remphan1618")

# Install required packages
!pip install -q iopaint pyngrok huggingface_hub gfpgan rembg onnxruntime
print("✅ Required packages installed")

# Define base directories - Using /content/ for user accessibility
BASE_MODEL_DIR = "/content/iopaint/models"
TORCH_HUB_DIR = os.path.join(BASE_MODEL_DIR, "torch/hub/checkpoints")
os.makedirs(BASE_MODEL_DIR, exist_ok=True)
os.makedirs(TORCH_HUB_DIR, exist_ok=True)

# Create a symlink from default location to our accessible location
default_model_dir = "/root/.cache/iopaint/models"
os.makedirs(os.path.dirname(default_model_dir), exist_ok=True)
if os.path.exists(default_model_dir):
    if os.path.islink(default_model_dir):
        os.unlink(default_model_dir)
    else:
        import shutil
        print(f"Moving existing models from {default_model_dir} to {BASE_MODEL_DIR}...")
        for item in os.listdir(default_model_dir):
            src = os.path.join(default_model_dir, item)
            dst = os.path.join(BASE_MODEL_DIR, item)
            if not os.path.exists(dst):
                if os.path.isdir(src):
                    shutil.copytree(src, dst)
                else:
                    shutil.copy2(src, dst)
        shutil.rmtree(default_model_dir)

# Create symlink
os.symlink(BASE_MODEL_DIR, default_model_dir)
print(f"✅ Models directory set to {BASE_MODEL_DIR} (accessible via /content/)")
print(f"✅ Created symlink from {default_model_dir} to {BASE_MODEL_DIR}")

# ========== HANDLE NGROK TOKEN ==========
# Load token from file if exists
if not ngrok_token and os.path.exists(token_file):
    with open(token_file, 'r') as f:
        ngrok_token = f.read().strip()
    print(f"✅ Loaded Ngrok token from {token_file}")

# Save token if requested
if ngrok_token and save_token:
    with open(token_file, 'w') as f:
        f.write(ngrok_token)
    os.chmod(token_file, 0o600)  # Set permissions to user-only
    print(f"✅ Saved Ngrok token to {token_file}")

if not ngrok_token:
    print("❌ ERROR: Ngrok token is required!")
    print("Please get your token from https://dashboard.ngrok.com/get-started/your-authtoken")
    sys.exit(1)

# ========== CHECKPOINT MODEL DOWNLOADER ==========
# Checkpoint models database
CHECKPOINT_MODELS = {
    "lama": {
        "url": "https://github.com/Sanster/models/releases/download/add_big_lama/big-lama.pt",
        "filename": "big-lama.pt",
        "size_mb": 196,
        "description": "Default fast inpainting model"
    },
    "mat": {
        "url": "https://github.com/Sanster/models/releases/download/mat/Places_512_FullData_G.pth",
        "filename": "Places_512_FullData_G.pth",
        "size_mb": 675,
        "description": "Better quality than LaMa but slower"
    },
    "sam2_base": {
        "url": "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt",
        "filename": "sam2_hiera_base_plus.pt",
        "size_mb": 309,
        "description": "SAM2 model for interactive segmentation"
    },
    "gfpgan": {
        "url": "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth",
        "filename": "GFPGANv1.3.pth",
        "size_mb": 332,
        "description": "Face restoration model"
    },
    "migan": {
        "url": "https://github.com/Sanster/models/releases/download/migan/migan_patch.pt",
        "filename": "migan_patch.pt",
        "size_mb": 395,
        "description": "Another good inpainting model"
    }
}

def download_checkpoint(url, dest_path):
    """Download a file with progress bar"""
    if os.path.exists(dest_path):
        file_size_mb = os.path.getsize(dest_path) / (1024 * 1024)
        print(f"✅ Already exists: {os.path.basename(dest_path)} ({file_size_mb:.1f} MB)")
        return True

    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))

            with open(dest_path, 'wb') as f, tqdm(
                desc=os.path.basename(dest_path),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for chunk in r.iter_content(chunk_size=8192):
                    size = f.write(chunk)
                    bar.update(size)

        if os.path.exists(dest_path) and os.path.getsize(dest_path) > 0:
            size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"✅ Downloaded: {os.path.basename(dest_path)} ({size_mb:.1f} MB)")
            return True
        else:
            print(f"❌ Failed to download: {dest_path}")
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Process checkpoint models
if checkpoint_models:
    model_list = [m.strip() for m in checkpoint_models.split(",") if m.strip()]
    print(f"\n📦 Downloading {len(model_list)} checkpoint models: {', '.join(model_list)}")

    for model_name in model_list:
        if model_name in CHECKPOINT_MODELS:
            print(f"\n📥 Downloading {model_name}: {CHECKPOINT_MODELS[model_name]['description']}")
            dest_path = os.path.join(TORCH_HUB_DIR, CHECKPOINT_MODELS[model_name]['filename'])
            download_checkpoint(CHECKPOINT_MODELS[model_name]['url'], dest_path)
        else:
            print(f"❌ Unknown model: {model_name}")

# ========== DIFFUSERS MODEL DOWNLOADER ==========
def sanitize_name(repo_id):
    """Create a valid model name from repository ID"""
    name = repo_id.split("/")[-1].lower()
    name = name.replace("-", "_").replace(".", "_").replace(" ", "_")
    if len(name) > 30:
        name = name[:30]
    return name

def setup_diffusers_model(repo_id, use_fp16=False, pre_download=False):
    """Set up a diffusers model by creating model_info.json"""
    model_name = sanitize_name(repo_id)
    model_dir = os.path.join(BASE_MODEL_DIR, model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Create model_info.json
    config = {
        "name": model_name,
        "repo_id": repo_id,
        "subfolder": None,
        "type": "diffusers",
        "fp16": use_fp16,
        "revision": "main",
        "use_checkpoint": False  # Force diffusers format only
    }

    config_path = os.path.join(model_dir, "model_info.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=2)

    print(f"✅ Set up model: {model_name}")
    print(f"  • Repository: {repo_id}")
    print(f"  • Using FP16: {'Yes' if use_fp16 else 'No'}")

    # Optionally pre-download the model
    if pre_download:
        try:
            print(f"  • Pre-downloading diffusers components...")

            # Import huggingface_hub
            from huggingface_hub import snapshot_download

            # Only download diffusers components
            allow_patterns = [
                "*.json",
                "scheduler/**",
                "text_encoder/**",
                "text_encoder_2/**",
                "tokenizer/**",
                "unet/**",
                "vae/**",
                "safety_checker/**",
                "feature_extractor/**"
            ]

            # Skip checkpoint files
            ignore_patterns = ["*.safetensors", "*.ckpt", "*.bin", "*.pt"]

            # Download with progress tracking
            print(f"    ⏳ Downloading from {repo_id}...")
            snapshot_download(
                repo_id=repo_id,
                local_dir=model_dir,
                allow_patterns=allow_patterns,
                ignore_patterns=ignore_patterns
            )
            print(f"    ✅ Download complete!")
        except Exception as e:
            print(f"    ❌ Error pre-downloading: {str(e)}")
            print(f"    ℹ️ Model will download when first used in IOpaint.")
    else:
        print(f"  • Model will download when first used in IOpaint")

    return model_name

# Process diffusers models
configured_diffusers = []
if diffusers_models:
    model_repos = [m.strip() for m in diffusers_models.split(",") if m.strip()]
    print(f"\n🧩 Setting up {len(model_repos)} diffusers models")

    for repo_id in model_repos:
        print(f"\n📥 Processing: {repo_id}")
        model_name = setup_diffusers_model(repo_id, use_fp16_for_diffusers, pre_download_diffusers)
        configured_diffusers.append(model_name)
        time.sleep(0.5)  # Small delay between setups

# ========== NGROK & IOPAINT LAUNCHER ==========
from pyngrok import ngrok

# Clear any existing tunnels
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    ngrok.disconnect(tunnel.public_url)

# Set auth token
ngrok.set_auth_token(ngrok_token)

# Create tunnel
url = ngrok.connect(addr=port, bind_tls=True)
print(f"\n🌎 IOPaint URL: {url}")

# Display clickable link and file explorer info
display(HTML(f'''
<div style="background-color: #f0f5ff; padding: 15px; border-radius: 10px; margin: 10px 0;">
  <h3>🎨 IOpaint is Ready!</h3>
  <p><b>Access URL:</b> <a href="{url}" target="_blank">{url}</a></p>
  <p><b>Models folder:</b> <code>/content/iopaint/models</code> (accessible in file explorer)</p>
  <p><b>Selected model:</b> <code>{selected_model}</code></p>
</div>
'''))

# Build IOPaint command
cmd = [
    'iopaint', 'start',
    '--model', selected_model,
    '--device', 'cuda',
    '--port', str(port),
    '--host', '0.0.0.0'
]

# Add optional parameters
if enable_interactive_seg:
    cmd.extend(['--enable-interactive-seg', '--interactive-seg-device', 'cuda'])

if enable_remove_bg:
    cmd.append('--enable-remove-bg')

if enable_realesrgan:
    cmd.extend(['--enable-realesrgan', '--realesrgan-device', 'cuda'])

if enable_gfpgan:
    cmd.extend(['--enable-gfpgan', '--gfpgan-device', 'cuda'])

if disable_nsfw:
    cmd.append('--disable-nsfw-checker')

print("\n🚀 Launching IOPaint with command:")
print(' '.join(cmd))
print("\n⏳ Please wait until you see 'Application startup complete'...\n")

# Run the process and keep it alive
async def run_process():
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    async def read_stream(stream):
        while True:
            line = await stream.readline()
            if not line:
                break
            print(line.decode('utf-8').strip())

    await asyncio.gather(
        read_stream(process.stdout),
        read_stream(process.stderr)
    )

# Run the process
try:
    await run_process()
except KeyboardInterrupt:
    print("Stopping on user request...")
finally:
    print("Cleaning up Ngrok tunnels...")
    ngrok.kill()